In [2]:
!unzip "/content/drive/MyDrive/Biz data/IMDB Dataset.csv.zip"

Archive:  /content/drive/MyDrive/Biz data/IMDB Dataset.csv.zip
  inflating: IMDB Dataset.csv        


In [3]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/colab_setup.sh -O - | bash
!pip install pyspark spark-nlp findspark

--2021-07-25 13:08:04--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-07-25 13:08:04 (28.5 MB/s) - written to stdout [1608/1608]

setup Colab for PySpark 3.0.2 and Spark NLP 3.1.0
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 k

In [4]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import numpy as np
import pandas as pd
import sparknlp
from sparknlp.annotator import *

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import findspark

spark = sparknlp.start(gpu=True)
#spark = SparkSession.Builder().getOrCreate()
spark

In [5]:
df2 = pd.read_csv('/content/IMDB Dataset.csv')
df=spark.createDataFrame(df2) 
df.printSchema()
df.describe().show()
df.show()

root
 |-- review: string (nullable = true)
 |-- sentiment: string (nullable = true)

+-------+--------------------+---------+
|summary|              review|sentiment|
+-------+--------------------+---------+
|  count|               50000|    50000|
|   mean|                null|     null|
| stddev|                null|     null|
|    min|A Turkish Bat...| negative|
|    max|ý thýnk uzak ýs t...| positive|
+-------+--------------------+---------+

+--------------------+---------+
|              review|sentiment|
+--------------------+---------+
|One of the other ...| positive|
|A wonderful littl...| positive|
|I thought this wa...| positive|
|Basically there's...| negative|
|Petter Mattei's "...| positive|
|Probably my all-t...| positive|
|I sure would like...| positive|
|This show was an ...| negative|
|Encouraged by the...| negative|
|If you like origi...| positive|
|Phil the Alien is...| negative|
|I saw this movie ...| negative|
|So im not a big f...| negative|
|The cast played 

In [6]:
user_regex = r"(@\w{1,15})"
hashtag_replace_regex = "#(\w{1,})"
url_regex = r"https?://\S+|www\.\S+"
email_regex = r"[\w.-]+@[\w.-]+\[a-zA-Z]{1,}"
bracket_regex = r"<br />"
end_line =r"\n"
punctuation=r'[^\sa-zA-Z0-9]'
special_char=r"\\W"

In [7]:
from pyspark.sql.functions import regexp_replace
result = (
    df
    .withColumn("review",f.regexp_replace(f.col("review"),bracket_regex," "))
    .withColumn("review",f.regexp_replace(f.col("review"),url_regex," "))
    .withColumn("review",f.regexp_replace(f.col("review"),special_char," "))
    .withColumn("review",f.regexp_replace(f.col("review"),punctuation," "))
    .withColumn("review",regexp_replace("review","\n",''))
    .withColumn("sentiment",regexp_replace("sentiment","positive",'1'))
    .withColumn("sentiment",regexp_replace("sentiment","negative",'0'))
    
)

In [8]:
result=result.withColumn("sentiment",result.sentiment.cast('int'))
result.toPandas()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming t...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there s a family where a little boy ...,0
4,Petter Mattei s Love in the Time of Money is...,1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,Bad plot bad dialogue bad acting idiotic di...,0
49997,I am a Catholic taught in parochial elementary...,0
49998,I m going to have to disagree with the previou...,0


In [9]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
train, test = result.randomSplit([0.8, 0.2], seed=12345)

In [10]:
from sparknlp.base import *
document = DocumentAssembler()\
      .setInputCol("review")\
      .setOutputCol("document")
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normal")
bert_embeddings =  DistilBertEmbeddings.pretrained("distilbert_base_uncased", "en")\
    .setInputCols(["document",'normal'])\
    .setOutputCol("embeddings")
embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("sentiment")\
    .setMaxEpochs(150)\
    .setLr(0.0022)\
    .setBatchSize(128)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('logs')
docClassifier = SentimentDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("label") \
    .setLabelColumn("sentiment") \
    .setBatchSize(128) \
    .setMaxEpochs(150) \
    .setLr(0.001) \
    .setDropout(0.5) \
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('logs')
bert_clf_pipeline = Pipeline(stages=[
    document, 
    tokenizer,
    normalizer,
    bert_embeddings,
    embeddingsSentence,
    classsifierdl
])

distilbert_base_uncased download started this may take some time.
Approximate size to download 236 MB
[OK!]


In [11]:
df_bert = bert_clf_pipeline.fit(train)

In [12]:
from sklearn.metrics import classification_report

preds = df_bert.transform(test)

preds_df = preds.select('review','sentiment',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [13]:
print (classification_report(preds_df['result'].astype("int64"), preds_df['sentiment']))

              precision    recall  f1-score   support

           0       0.82      0.83      0.82      5007
           1       0.83      0.82      0.83      5115

    accuracy                           0.82     10122
   macro avg       0.82      0.82      0.82     10122
weighted avg       0.82      0.82      0.82     10122



In [14]:
!cat /content/logs/ClassifierDLApproach_dfa2419d3cb1.log

Training started - epochs: 150 - learning_rate: 0.0022 - batch_size: 128 - training_examples: 39878 - classes: 2
Epoch 0/150 - 4.07s - loss: 169.98425 - acc: 0.780833 - batches: 312
Epoch 1/150 - 1.43s - loss: 158.78099 - acc: 0.81273615 - batches: 312
Epoch 2/150 - 1.41s - loss: 157.56769 - acc: 0.81580085 - batches: 312
Epoch 3/150 - 1.39s - loss: 157.35054 - acc: 0.8179612 - batches: 312
Epoch 4/150 - 1.49s - loss: 157.3409 - acc: 0.82007134 - batches: 312
Epoch 5/150 - 1.37s - loss: 157.27095 - acc: 0.8216245 - batches: 312
Epoch 6/150 - 1.38s - loss: 157.12538 - acc: 0.8227298 - batches: 312
Epoch 7/150 - 1.37s - loss: 156.86514 - acc: 0.82293075 - batches: 312
Epoch 8/150 - 1.36s - loss: 156.57451 - acc: 0.82390624 - batches: 312
Epoch 9/150 - 1.38s - loss: 156.39667 - acc: 0.82455933 - batches: 312
Epoch 10/150 - 1.38s - loss: 156.27022 - acc: 0.825313 - batches: 312
Epoch 11/150 - 1.38s - loss: 156.1734 - acc: 0.8258405 - batches: 312
Epoch 12/150 - 1.37s - loss: 156.12346 - ac